# Visualização de Queries do Banco de Dados

## Configuração do Ambiente

Célula inicial para importar as bibliotecas necessárias e estabelecer a conexão com o banco de dados DuckDB.

In [19]:
import duckdb
import pandas as pd

# Conecta ao banco de dados persistente
con = duckdb.connect(database='BancoNaoRelacional.db', read_only=True)

# Configura o pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

---

## Query 1: Relação entre Desempenho no ENEM e Remuneração

**Objetivo:** Analisar a relação entre o desempenho médio dos alunos no ENEM e sua remuneração média após ingressarem no mercado de trabalho.

**Lógica:** A query associa o ano de admissão do aluno no mercado de trabalho (`data_admissao_declarada`) com o ano do ENEM (`nu_ano`), assumindo que ambos representam o mesmo ciclo de formação. A análise é focada em uma escola específica.

In [24]:
query1 = """
WITH medias_salariais AS (
    -- Calcula a média da remuneração nominal e do salário contratual dos alunos por:
    -- 1. ano de admissão no trabalho (extraído de data_admissao_declarada)
    -- 2. escola em que o aluno estudou (via co_escola_educacenso na fato_aluno_trajetoria)
    SELECT
        EXTRACT(YEAR FROM data_admissao_declarada) AS ano_admissao,
        co_escola_educacenso,
        ROUND(AVG(vl_remun_media_nom), 2) AS media_remun_media_nom,
        ROUND(AVG(vl_salario_contratual), 2) AS media_salario_contratual
    FROM
        fato_aluno_trajetoria
    WHERE
        data_admissao_declarada IS NOT NULL
    GROUP BY
        ano_admissao,
        co_escola_educacenso
)

-- Associa as médias salariais calculadas com as médias de desempenho no ENEM 
-- para os mesmos anos e escolas, retornando os indicadores lado a lado.
SELECT
    ms.ano_admissao,
    ms.co_escola_educacenso,
    ms.media_remun_media_nom,
    ms.media_salario_contratual,
    de.nu_media_cn_enem,
    de.nu_media_ch_enem,
    de.nu_media_lp_enem,
    de.nu_media_mt_enem,
    de.nu_media_red_enem
FROM
    medias_salariais ms
JOIN dim_escola_indicadores_anuais de
    ON ms.ano_admissao = de.nu_ano
    AND ms.co_escola_educacenso = de.co_escola_educacenso
WHERE
    ms.co_escola_educacenso = '53006178'  -- IFB (Instituto Federal de Brasília)
ORDER BY
    ms.ano_admissao;
"""

df1 = con.execute(query1).df()
print("Resultado da Query 1:")
display(df1)

Resultado da Query 1:


,ano_admissao,co_escola_educacenso,media_remun_media_nom,media_salario_contratual,nu_media_cn_enem,nu_media_ch_enem,nu_media_lp_enem,nu_media_mt_enem,nu_media_red_enem
0,2010,53006178,6744.55,5373.87,545.00,565.25,497.55,516.75,593.18
1,2011,53006178,4595.93,3255.63,464.68,480.41,505.84,490.82,454.38
2,2012,53006178,4412.21,3396.07,509.25,546.81,497.48,544.22,527.39
3,2013,53006178,4120.08,2744.08,518.91,524.60,494.21,574.02,552.22
4,2014,53006178,3523.31,2625.32,523.88,562.02,513.67,514.62,505.33
5,2015,53006178,3040.17,2004.31,506.86,558.60,499.47,478.62,544.00


---

## Query 2: Correlação entre Salário e Desempenho no ENEM por Município

**Objetivo:** Analisar a correlação entre o salário médio dos egressos e o desempenho médio no ENEM das escolas localizadas no mesmo município do vínculo empregatício.

**Lógica:** A query calcula a média salarial por município e ano de admissão, e a média de notas do ENEM por município e ano. Em seguida, junta os resultados para comparação.

**Correção:** A query original usava colunas `id_loc_vinculo` e `id_loc_escola` que não existem. A versão corrigida junta os dados usando `municipio` e `uf`.

In [25]:
query2 = """
WITH vinculos_por_localizacao AS (
    -- Agrupa os egressos por ano de admissão e localização do vínculo, calculando o salário médio.
    SELECT
        EXTRACT(YEAR FROM data_admissao_declarada) AS ano_admissao,
        uf_vinculo,
        municipio_vinculo,
        AVG(vl_salario_contratual) AS media_salarial_local
    FROM
        fato_aluno_trajetoria
    WHERE
        data_admissao_declarada IS NOT NULL AND municipio_vinculo IS NOT NULL
    GROUP BY
        ano_admissao, uf_vinculo, municipio_vinculo
),
enem_por_localizacao AS (
    -- Calcula a média geral do ENEM para cada escola por ano e localização.
    SELECT
        nu_ano,
        uf_escola,
        municipio_escola,
        -- Calcula a média das 5 notas do ENEM
        AVG((nu_media_cn_enem + nu_media_ch_enem + nu_media_lp_enem + nu_media_mt_enem + nu_media_red_enem) / 5) AS media_geral_enem_escola
    FROM
        dim_escola_indicadores_anuais
    WHERE municipio_escola IS NOT NULL
    GROUP BY
        nu_ano, uf_escola, municipio_escola
)
-- Junta os dados de salários e desempenho no ENEM pela localização e ano.
SELECT
    vpl.ano_admissao,
    vpl.uf_vinculo,
    vpl.municipio_vinculo,
    ROUND(vpl.media_salarial_local, 2) AS media_salarial,
    ROUND(epl.media_geral_enem_escola, 2) AS media_enem
FROM
    vinculos_por_localizacao vpl
JOIN
    enem_por_localizacao epl ON vpl.municipio_vinculo = epl.municipio_escola 
                            AND vpl.uf_vinculo = epl.uf_escola 
                            AND vpl.ano_admissao = epl.nu_ano
ORDER BY
    vpl.uf_vinculo ASC, vpl.municipio_vinculo ASC;
"""

df2 = con.execute(query2).df()
print("Resultado da Query 2:")
display(df2)

Resultado da Query 2:


,ano_admissao,uf_vinculo,municipio_vinculo,media_salarial,media_enem
0,2014,AL,Arapiraca,1243.05,510.58
1,2012,AL,Arapiraca,2200.44,521.72
2,2010,AL,Arapiraca,16591.91,527.62
3,2012,AL,Maceió,8973.18,522.95
4,2013,AL,Pilar,3482.36,462.12
...,...,...,...,...,...
196,2012,SP,São Paulo,2519.93,563.45
197,2009,SP,São Paulo,5708.23,536.35
198,2015,TO,Palmas,1459.26,540.51
199,2014,TO,Palmas,4094.27,526.99


---

## Query 3: Características do Mercado de Trabalho vs. Desempenho no ENEM

**Objetivo:** Analisar a relação entre as características do mercado de trabalho local (total de vínculos, horas, remuneração) e o desempenho médio no ENEM das escolas na mesma localidade.

**Lógica:** A query agrega dados de vínculos ativos por localidade e a média do ENEM por localidade, depois os junta para análise.

**Correção:** A query original usava colunas `id_loc_vinculo`, `id_vinculo` e `id_loc_escola` que não existem. A versão corrigida junta os dados usando `municipio` e `uf` e agrupa por eles.

In [26]:
query3 = """
WITH vinculos_por_localizacao AS (
    -- Agrega dados de vínculos empregatícios por localização (município/UF).
    -- Calcula o total de vínculos, média de horas contratuais e remuneração média.
    SELECT
        uf_vinculo,
        municipio_vinculo,
        COUNT(*) AS total_vinculos,
        AVG(qtd_hora_contr) AS media_horas_contratuais,
        AVG(vl_remun_media_nom) AS remuneracao_media_nominal
    FROM
        fato_aluno_trajetoria
    WHERE
        indicador_vinculo_ativo = TRUE
        AND qtd_hora_contr > 0
        AND vl_remun_media_nom > 0
        AND municipio_vinculo IS NOT NULL
    GROUP BY
        uf_vinculo, municipio_vinculo
    HAVING
        COUNT(*) >= 5
),
media_enem_por_localizacao AS (
    -- Calcula a média geral do ENEM e a quantidade de escolas por localização.
    SELECT
        uf_escola,
        municipio_escola,
        AVG((nu_media_cn_enem + nu_media_ch_enem + nu_media_lp_enem + nu_media_mt_enem + nu_media_red_enem) / 5) AS media_enem,
        COUNT(DISTINCT co_escola_educacenso) AS qtd_escolas
    FROM
        dim_escola_indicadores_anuais
    WHERE municipio_escola IS NOT NULL
    GROUP BY
        uf_escola, municipio_escola
)
-- Junta os dados de vínculos e de desempenho no ENEM pela localização.
SELECT
    vpl.uf_vinculo AS uf,
    vpl.municipio_vinculo AS municipio,
    vpl.total_vinculos,
    ROUND(vpl.media_horas_contratuais, 2) AS media_horas_contratuais,
    ROUND(vpl.remuneracao_media_nominal, 2) AS remuneracao_media_nominal,
    -- Calcula o valor médio por hora contratual
    ROUND(vpl.remuneracao_media_nominal / NULLIF(vpl.media_horas_contratuais, 0), 2) AS valor_hora_medio,
    ROUND(mepl.media_enem, 2) AS media_enem,
    mepl.qtd_escolas
FROM
    vinculos_por_localizacao vpl
JOIN
    media_enem_por_localizacao mepl ON vpl.municipio_vinculo = mepl.municipio_escola AND vpl.uf_vinculo = mepl.uf_escola
ORDER BY
    vpl.uf_vinculo, remuneracao_media_nominal DESC;
"""

df3 = con.execute(query3).df()
print("Resultado da Query 3:")
display(df3)

Resultado da Query 3:


,uf,municipio,total_vinculos,media_horas_contratuais,remuneracao_media_nominal,valor_hora_medio,media_enem,qtd_escolas
0,AL,Arapiraca,5,38.00,5694.38,149.85,519.46,29
1,AL,Maceió,8,38.13,4586.18,120.29,516.75,131
2,AM,Manaus,5,34.60,3487.40,100.79,507.03,170
3,AP,Macapá,5,38.80,9400.77,242.29,494.55,43
4,BA,Salvador,44,38.32,3984.37,103.98,531.24,247
...,...,...,...,...,...,...,...,...
61,SP,Ribeirão Preto,5,39.80,2533.77,63.66,559.11,71
62,SP,Barueri,37,39.35,2189.37,55.64,546.30,34
63,SP,Lucélia,5,42.40,1839.03,43.37,528.19,2
64,SP,Guarulhos,6,38.00,1490.47,39.22,530.97,159


---

## Query 4: Persistência no Emprego vs. Taxa de Abandono Escolar

**Objetivo:** Comparar a "persistência no emprego" (tempo de emprego) dos egressos com a "taxa de abandono" da instituição no ano em que ingressaram.

**Lógica:** A query foca em alunos de uma escola específica que ingressaram entre 2009 e 2015. Ela junta os dados do aluno com os indicadores da escola no ano de ingresso para obter a taxa de abandono e, em seguida, categoriza tanto o tempo de emprego quanto a taxa de abandono em faixas.

In [27]:
query4 = """
WITH dados_analise AS (
    -- Seleciona os dados de alunos que entraram em um período específico e tiveram um vínculo empregatício encerrado.
    -- Junta com os indicadores da escola para obter a taxa de abandono do ano de ingresso.
    SELECT
        fat.id_aluno,
        fat.ano_entrada_ifb,
        fat.tempo_emprego,
        deia.nu_taxa_abandono
    FROM
        fato_aluno_trajetoria fat
    JOIN
        dim_escola_indicadores_anuais deia ON fat.co_escola_educacenso = deia.co_escola_educacenso
                                           AND fat.ano_entrada_ifb = deia.nu_ano
    WHERE
        fat.ano_entrada_ifb BETWEEN 2009 AND 2015
        -- Filtra por um campus específico do IFB, como no script original
        AND fat.co_escola_educacenso = '53006178'
        -- Considera apenas vínculos que foram desligados
        AND fat.motivo_desligamento IS NOT NULL AND fat.motivo_desligamento != 'NAO DESLIGADO NO ANO'
        AND fat.tempo_emprego IS NOT NULL
        AND deia.nu_taxa_abandono IS NOT NULL
)

-- Categoriza o tempo de emprego e a taxa de abandono em faixas (BAIXA, MEDIA, ALTA)
SELECT
    id_aluno,
    ano_entrada_ifb,
    CASE
        WHEN tempo_emprego <= 12 THEN 'BAIXA'
        WHEN tempo_emprego > 12 AND tempo_emprego <= 36 THEN 'MEDIA'
        ELSE 'ALTA'
    END AS persistencia_emprego,
    CASE
        WHEN nu_taxa_abandono < 5 THEN 'BAIXA'
        WHEN nu_taxa_abandono >= 5 AND nu_taxa_abandono < 10 THEN 'MEDIA'
        ELSE 'ALTA'
    END AS categoria_abandono_escolar
FROM
    dados_analise;
"""

df4 = con.execute(query4).df()
print("Resultado da Query 4:")
display(df4)

Resultado da Query 4:


,id_aluno,ano_entrada_ifb,persistencia_emprego,categoria_abandono_escolar
0,14,2013,BAIXA,MEDIA
1,18,2015,ALTA,BAIXA
2,45,2012,BAIXA,ALTA
3,53,2015,MEDIA,BAIXA
4,56,2015,BAIXA,BAIXA
...,...,...,...,...
1607,22329,2013,BAIXA,MEDIA
1608,8401,2012,BAIXA,ALTA
1609,3546,2012,BAIXA,ALTA
1610,20400,2015,BAIXA,BAIXA


---

## Query 5: Mobilidade de Alunos para Emprego

**Objetivo:** Identificar alunos de cursos presenciais que conseguiram um emprego em um município diferente daquele onde estudaram.

**Lógica:** A query filtra por alunos de cursos presenciais com remuneração anual acima de R$1000 e compara o município da escola com o município do vínculo empregatício.

In [28]:
query5 = """
SELECT
    fat.id_aluno,
    fat.vl_ultima_remuneracao_ano AS ultima_remuneracao_anual,
    fat.no_curso,
    fat.modalidade_ensino,
    deia.municipio_escola,
    fat.municipio_vinculo AS municipio_empregado
FROM
    fato_aluno_trajetoria fat
JOIN
    dim_escola_indicadores_anuais deia ON fat.co_escola_educacenso = deia.co_escola_educacenso
                                       AND fat.ano_entrada_ifb = deia.nu_ano
WHERE
    fat.modalidade_ensino = 'Educação Presencial'
    AND fat.vl_ultima_remuneracao_ano >= 1000
    AND deia.municipio_escola != fat.municipio_vinculo
    -- Garante que as localidades são válidas para comparação
    AND deia.municipio_escola IS NOT NULL
    AND fat.municipio_vinculo IS NOT NULL
ORDER BY
    ultima_remuneracao_anual DESC;
"""

df5 = con.execute(query5).df()
print("Resultado da Query 5:")
display(df5)

Resultado da Query 5:


,id_aluno,ultima_remuneracao_anual,no_curso,modalidade_ensino,municipio_escola,municipio_empregado
0,7925,47282.95,ORÇAMENTO PÚBLICO,Educação Presencial,Brasília,Curitiba
1,17997,38132.64,PROCESSO INTERNO DE LICITAÇÃO,Educação Presencial,Brasília,Uberaba
2,8180,37273.11,TÉCNICO EM AGROPECUÁRIA,Educação Presencial,Brasília,Planaltina
3,15357,26362.63,DIDÁTICA DE FACILITADORES,Educação Presencial,Brasília,Uberaba
4,3031,24887.86,ESPANHOL NÍVEL BÁSICO II,Educação Presencial,Brasília,Goiânia
...,...,...,...,...,...,...
862,15336,1008.22,TÉCNICO EM AGROPECUÁRIA,Educação Presencial,Brasília,Goiânia
863,10198,1004.40,TÉCNICO EM LOGÍSTICA,Educação Presencial,Brasília,Recife
864,18740,1000.00,OPERADOR DE MICROCOMPUTADOR,Educação Presencial,Brasília,Águas Lindas de Goiás
865,18305,1000.00,PINTURA / TEXTURIZAÇÃO,Educação Presencial,Brasília,Águas Lindas de Goiás


---

## Finalização

Fecha a conexão com o banco de dados.

In [29]:
con.close()